In [1]:
# https://github.com/ibis-project/ibis/pull/2112
from typing import Dict, Optional
import ibis
import pandas as pd

In [2]:
db_conf = dict(
    host='localhost', port='6274', 
    user='admin', password='HyperInteractive', 
    database='ibis_testing'
)
con = ibis.omniscidb.connect(**db_conf)

con.list_tables()

['diamonds', 'batting', 'awards_players', 'functional_alltypes', 'geo']

In [3]:
def update(
    con, 
    table_name: str, 
    condition: ibis.expr.types.Expr, 
    input_expr: Optional[Dict[str, ibis.expr.types.Expr]] is None,
    input_data: pd.DataFrame is None
):
    """
    Update method.

    Parameters
    ----------
    table_name : str
    condition : ibis.expr.types.Expr
    input_expr : dict of ibis.expr.types.Expr, optional, default None
    input_data : pandas.DataFrame, optional, default None
    """
    if input_expr is None and input_data is None:
        raise Exception('No set clause given.')
    
    # setup
    t = con.table(table_name)
    
    cond_str = t[condition].compile()
    cond_str = cond_str[cond_str.index('WHERE'):]

    update_stmt = ''
    update_tmpl = 'UPDATE {}'.format(table_name) + ' SET {} {};\n'
    
    update_set = {}
    series = {}
    
    size = t[cond].count().execute()
    print('series size expected:', size)
    
    # create update statement for input_expr
    if input_expr is not None:
        for k, v in input_expr.items():
            if not isinstance(v, ibis.expr.types.Expr):
                raise Exception(
                    'Data expression format expected: ibis.expr.types.Expr. '
                    'Data format given: {}'.format(type(v))
                )

            value = v.compile()
            try:
                i_end = value.index('FROM')
                i_end -= 8
            except ValueError:
                i_end = -7
            value = value[7:i_end]
            update_set[k] = value

        update_stmt = update_tmpl.format(
            ', '.join(['{}=({})'.format(k, v) for k, v in update_set.items()]),
            cond_str
        )
        
    # create update statement for input_data
    if input_data is not None:
        key_name = input_data.index.name
        if cond_str == '':
            cond_str = ' WHERE 1=1 AND'
        for i, row in input_data.iterrows():
            update_stmt += update_tmpl.format(
                ', '.join(['"{}"=({})'.format(k, repr(v)) for k, v in row.items()]),
                cond_str + ' AND ("{}" = {})'.format(key_name, repr(i))
            )
            
        
    print('name:', t.name)
    print('cond:', cond_str)
    print('=' * 80)
    for stmt in update_stmt.split(';'):
        stmt = stmt.strip().replace('\n', '')
        if not stmt:
            continue
        print('>>>', stmt)
        con.con.execute(stmt)


# test data
table_name = 'functional_alltypes'
t = con.table(table_name)
cond = (5 <= t.index) & (t.index < 15)

df = t[cond].execute()
# set index, used as primary key
df.set_index('index', inplace=True, drop=False)

# prepare new data
string_se = pd.Series([str(x) for x in range(1000, 1010)])
string_se.index = df.index
df['string_col'] = string_se

update(
    con, 
    table_name=table_name,
    condition=cond,
    input_expr={'int_col':t.index+1},
    input_data=df[['string_col']],
)

series size expected: 10
name: functional_alltypes
cond: WHERE ("index" >= 5) AND
      ("index" < 15)
>>> UPDATE functional_alltypes SET int_col=("index" + 1) WHERE ("index" >= 5) AND      ("index" < 15)
>>> UPDATE functional_alltypes SET "string_col"=('1000') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 5)
>>> UPDATE functional_alltypes SET "string_col"=('1001') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 6)
>>> UPDATE functional_alltypes SET "string_col"=('1002') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 7)
>>> UPDATE functional_alltypes SET "string_col"=('1003') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 8)
>>> UPDATE functional_alltypes SET "string_col"=('1004') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 9)
>>> UPDATE functional_alltypes SET "string_col"=('1005') WHERE ("index" >= 5) AND      ("index" < 15) AND ("index" = 10)
>>> UPDATE functional_alltypes SET "string_col"=('1006') WHERE ("index" >=

In [4]:
df_check = t[cond].execute()
# set index, used as primary key
df_check.set_index('index', inplace=True, drop=False)
assert all(df_check['string_col'] == string_se)